<a href="https://colab.research.google.com/github/FraCorti/AI_Project/blob/master/spamFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Added **Kaggle** installation and downloading *spam.csv* dataset from the following link: [spam-collection-dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset)

In [1]:
!pip install kaggle
!mkdir /root/.kaggle
!echo '{"username":"fcorti","key":"fafa72652d9ae6ee3bab07bad5a1a96c"}' > /root/.kaggle/kaggle.json
!kaggle datasets download -d uciml/sms-spam-collection-dataset
!pip install pandas

  0% 0.00/208k [00:00<?, ?B/s]
100% 208k/208k [00:00<00:00, 63.5MB/s]


Imported **pandas** and convert the *sms-spam-collection-dataset.zip* file into *.csv* format


In [4]:
import pandas as pd
from zipfile import ZipFile
zip_file = ZipFile('/content/sms-spam-collection-dataset.zip')
fields= ['v1','v2'] 
data=pd.read_csv(zip_file.open('spam.csv'), usecols=fields, encoding = "latin-1")
print("Dataset shape: ", data.shape)



Dataset shape:  (5572, 2)


Import of  **Keras** 

In [0]:

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import metrics

# num_max = 1000
# le = LabelEncoder()
# tags = le.fit_transform(tags)
# tok = Tokenizer(num_words=num_max)
# tok.fit_on_texts(texts)
# mat_texts = tok.texts_to_matrix(texts,mode='count')

from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test = train_test_split(texts,tags, test_size = 0.2)
# mat_texts_tr = tok.texts_to_matrix(x_train,mode='count')
# mat_texts_tst = tok.texts_to_matrix(x_test,mode='count')

# max_len = 100
# x_train = tok.texts_to_sequences(x_train)
# x_test = tok.texts_to_sequences(x_test)
# cnn_texts_mat = sequence.pad_sequences(x_train,maxlen=max_len)
# max_len = 100
# cnn_texts_mat_tst = sequence.pad_sequences(x_test,maxlen=max_len)

NameError: ignored